In [34]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import copy
import pandas as pd
import time

from sklearn.decomposition import PCA

from utils.subspace_clustering_helper_funcs import *
from utils.preprocessing import *

- https://scikit-learn.org/stable/modules/classes.html#module-sklearn.manifold

## Loading in the data

In [35]:
## Pickle is theoretically faster for Python...

print("Loading")
start_time = time.time()
#data_df = pd.read_pickle('C:\\Users\\kdmen\\Desktop\\Research\\Data\\$M\\filtered_datasets\\metadata_IMU_EMG_allgestures_allusers.pkl')
data_df = pd.read_pickle('D:\\Kai_MetaGestureClustering_24\\saved_datasets\\filtered_datasets\\metadata_IMU_EMG_allgestures_allusers.pkl')
end_time = time.time()
print(f"Completed in {end_time - start_time}s")

Loading
Completed in 0.08206605911254883s


In [36]:
print(data_df.shape)
data_df.head()

(204800, 91)


,Participant,Gesture_ID,Gesture_Num,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,0.341797,-0.939941,0.000977,-0.007450,-0.192625,0.005321,-0.380859,...,0.000002,0.000002,0.000003,0.000020,0.000004,0.000004,0.000002,0.000009,0.000001,0.000002
1,P102,pan,1,0.336178,-0.963185,0.003898,0.009595,-0.190446,-0.026116,-0.394547,...,0.000003,0.000003,0.000003,0.000014,0.000007,0.000007,0.000002,0.000017,0.000001,0.000002
2,P102,pan,1,0.353539,-0.963704,0.011711,0.095966,-0.205480,-0.155563,-0.398406,...,0.000003,0.000003,0.000004,0.000007,0.000004,0.000005,0.000003,0.000020,0.000003,0.000002
3,P102,pan,1,0.352841,-0.950288,0.011509,0.058836,-0.184871,-0.083567,-0.389230,...,0.000003,0.000003,0.000006,0.000005,0.000004,0.000003,0.000004,0.000015,0.000003,0.000003
4,P102,pan,1,0.372621,-0.991273,0.029847,0.293946,-0.178756,-0.281361,-0.396043,...,0.000003,0.000002,0.000008,0.000003,0.000007,0.000022,0.000004,0.000017,0.000002,0.000003


In [37]:
metadata_cols = ['Participant', 'Gesture_ID', 'Gesture_Num']
metadata_cols_df = data_df[metadata_cols]
X = data_df.drop(metadata_cols, axis=1)

In [5]:
print(metadata_cols_df.shape)
metadata_cols_df.head()

(204800, 3)


,Participant,Gesture_ID,Gesture_Num
0,P102,pan,1
1,P102,pan,1
2,P102,pan,1
3,P102,pan,1
4,P102,pan,1


## Mean subtract each column, standardize each gesture

In [8]:
imu_split_df = X.iloc[:, :72]
emg_split_df = X.iloc[:, 72:]

ppd_emg_df = preprocess_df_by_gesture(emg_split_df, '$B', biosignal_switch_ix_lst=[])
ppd_imu_df = preprocess_df_by_gesture(imu_split_df, '$B', biosignal_switch_ix_lst=[])
X_ppd = pd.concat([metadata_cols_df, ppd_imu_df, ppd_emg_df], axis=1)

ppd_imu_df = pd.concat([metadata_cols_df, ppd_imu_df], axis=1)
ppd_emg_df = pd.concat([metadata_cols_df, ppd_emg_df], axis=1)

print(X_ppd.shape)
X_ppd.head()

(204800, 91)


,Participant,Gesture_ID,Gesture_Num,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,-0.551109,-0.738972,-0.985439,0.181924,0.059616,0.087024,1.055804,...,-0.276292,-0.026736,-0.873870,-1.036152,-0.580930,-0.719494,-0.502255,-1.750091,-0.127847,-0.094192
1,P102,pan,1,-0.571115,-0.821726,-0.975036,0.242607,0.067375,-0.024900,1.007074,...,-0.125822,0.089679,-0.816215,-2.082635,-0.006283,-0.139439,-0.367764,-0.208084,-0.111811,-0.039009
2,P102,pan,1,-0.509305,-0.823575,-0.947221,0.550111,0.013848,-0.485765,0.993332,...,-0.068451,0.117076,-0.668221,-3.403064,-0.526030,-0.478294,-0.300443,0.203266,0.113300,0.004728
3,P102,pan,1,-0.511788,-0.775810,-0.947939,0.417919,0.087222,-0.229441,1.026003,...,-0.058907,0.080977,-0.424416,-3.709413,-0.570894,-0.775155,-0.144710,-0.619539,0.146499,0.199975
4,P102,pan,1,-0.441369,-0.921726,-0.882652,1.254970,0.108993,-0.933639,1.001748,...,-0.003929,0.041526,-0.016530,-4.075150,-0.127710,2.682791,-0.141750,-0.208404,-0.035642,0.172662


In [9]:
print(ppd_imu_df.shape)
ppd_imu_df.head()

(204800, 75)


,Participant,Gesture_ID,Gesture_Num,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,...,IMU13_az,IMU13_vx,IMU13_vy,IMU13_vz,IMU15_ax,IMU15_ay,IMU15_az,IMU15_vx,IMU15_vy,IMU15_vz
0,P102,pan,1,-0.551109,-0.738972,-0.985439,0.181924,0.059616,0.087024,1.055804,...,-0.047646,-0.045230,0.104149,-0.046899,-0.027093,0.181512,0.003823,0.002525,0.055640,-0.009292
1,P102,pan,1,-0.571115,-0.821726,-0.975036,0.242607,0.067375,-0.024900,1.007074,...,-0.008147,-0.074827,0.006427,-0.049691,0.007038,0.184160,-0.014212,0.009704,0.034081,-0.026250
2,P102,pan,1,-0.509305,-0.823575,-0.947221,0.550111,0.013848,-0.485765,0.993332,...,-0.009861,-0.125477,0.203083,-0.095080,-0.077138,0.186451,0.034482,-0.061853,0.137769,-0.009996
3,P102,pan,1,-0.511788,-0.775810,-0.947939,0.417919,0.087222,-0.229441,1.026003,...,0.008394,-0.174411,-0.027033,-0.137434,-0.025918,0.193092,0.016815,-0.050325,-0.040597,-0.040180
4,P102,pan,1,-0.441369,-0.921726,-0.882652,1.254970,0.108993,-0.933639,1.001748,...,0.003086,-0.259050,0.118230,-0.111992,-0.081781,0.165432,0.029207,-0.197250,-0.006715,-0.030144


In [10]:
print(ppd_emg_df.shape)
ppd_emg_df.head()

(204800, 19)


,Participant,Gesture_ID,Gesture_Num,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,-0.362743,-0.801651,-0.383077,-0.195299,-0.203047,-0.464472,-0.276292,-0.026736,-0.873870,-1.036152,-0.580930,-0.719494,-0.502255,-1.750091,-0.127847,-0.094192
1,P102,pan,1,-0.351553,-0.775334,-0.382545,-0.154773,-0.131977,-0.295204,-0.125822,0.089679,-0.816215,-2.082635,-0.006283,-0.139439,-0.367764,-0.208084,-0.111811,-0.039009
2,P102,pan,1,-0.380825,-0.762588,-0.398388,-0.085411,0.017528,-0.205675,-0.068451,0.117076,-0.668221,-3.403064,-0.526030,-0.478294,-0.300443,0.203266,0.113300,0.004728
3,P102,pan,1,-0.366795,-0.765464,-0.374423,-0.073225,0.183172,0.009277,-0.058907,0.080977,-0.424416,-3.709413,-0.570894,-0.775155,-0.144710,-0.619539,0.146499,0.199975
4,P102,pan,1,-0.245578,-0.761283,-0.303976,-0.081947,0.224996,0.103319,-0.003929,0.041526,-0.016530,-4.075150,-0.127710,2.682791,-0.141750,-0.208404,-0.035642,0.172662


## Applying chosen dimensionality reduction algorithm:
> For now, just PCA

In [12]:
#df_t, dim_reduc_model = apply_dim_reduc(X_ms, model_str='PCA', num_dims=40, hp=None, modality=['EMG and IMU'], participant_inclusion=['All'], apply='ALL')
full_PCA40, dim_reduc_model = apply_dim_reduc(X_ppd, model_str='PCA', num_dims=40, use_full_dataset=True)

Start
Success


In [27]:
#pca = PCA(n_components=40)
#pca.fit(X)
#print(f"Total explained variance: {np.sum(pca.explained_variance_ratio_)}")

print(full_PCA40.shape)
full_PCA40.head()

(204800, 40)


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-0.030730,-0.052976,0.176556,-0.048078,-0.390790,-2.119829,0.875118,-0.187689,0.685004,0.870080,...,-0.253283,-0.248582,0.283079,-0.178321,-0.087101,-0.027081,0.000970,0.124380,-0.100580,-0.119294
1,0.077533,-0.108477,0.128970,-0.032167,-0.182756,-1.604412,0.701500,-0.068443,0.667338,1.098287,...,-0.423007,0.097453,0.184025,0.027788,0.112078,-0.414946,-0.210576,0.277156,0.051190,-0.039548
2,0.268279,-0.369027,0.268750,0.105239,-0.190292,-2.093812,1.012038,-0.080958,0.510502,0.758940,...,-0.311882,0.138655,-0.067485,0.201313,-0.051898,-0.479894,-0.079818,0.398142,0.058779,-0.095208
3,-0.106155,-0.291958,0.270681,0.035371,-0.204143,-2.314179,0.801514,-0.155267,0.519967,0.523683,...,-0.585487,-0.100021,0.082335,0.042349,-0.179251,-0.106235,-0.034940,0.386797,-0.034177,-0.132103
4,0.673338,-0.345606,0.246740,0.238184,0.948365,-0.041183,0.279347,0.034288,1.173712,2.115719,...,-0.262556,0.011353,-0.353489,0.209535,-0.402682,-0.353866,0.106203,0.366540,0.218785,-0.080414


In [14]:
dim_reduc_model.explained_variance_ratio_

array([0.19556914, 0.09905124, 0.07405321, 0.06060998, 0.05812266,
       0.04590841, 0.04439215, 0.03269277, 0.03184176, 0.02794496,
       0.02182327, 0.02065164, 0.01993278, 0.01934186, 0.0174894 ,
       0.01414135, 0.01335692, 0.01196144, 0.01162076, 0.01099271,
       0.01017437, 0.00974401, 0.0086075 , 0.00828047, 0.00651384,
       0.00624828, 0.00603505, 0.00595793, 0.0056693 , 0.00562337,
       0.0052174 , 0.00501087, 0.00485111, 0.004824  , 0.00451879,
       0.00432795, 0.00423068, 0.00403503, 0.00376879, 0.00351022])

First two PCs only account for 19.5% and 9.9% of the variance, respectively... plotting in 2D probably isn't worthwhile...

In [15]:
print(f"Total explained variance: {np.sum(dim_reduc_model.explained_variance_ratio_)}")

Total explained variance: 0.9486473799864418


In [17]:
full_PCA20, dim_reduc_model20 = apply_dim_reduc(X_ppd, model_str='PCA', num_dims=20, use_full_dataset=True)

Start
Success


In [25]:
full_PCA20.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.030730,-0.052977,0.176558,-0.048079,-0.390790,-2.119822,0.875129,-0.187667,0.685014,0.870081,-0.681692,0.068896,-0.521174,0.372720,-0.048495,0.936371,1.004633,0.317198,-0.312350,-0.237172
1,0.077533,-0.108477,0.128970,-0.032166,-0.182755,-1.604408,0.701508,-0.068431,0.667357,1.098277,-0.921642,0.263973,-0.636259,0.670122,0.126585,0.810550,1.463223,0.733985,-0.588444,-0.741753
2,0.268279,-0.369027,0.268750,0.105240,-0.190291,-2.093805,1.012053,-0.080945,0.510529,0.758957,-1.271628,-0.095496,-0.448483,0.881675,0.077877,0.500272,1.868600,0.919199,-0.623149,-1.604229
3,-0.106155,-0.291959,0.270682,0.035372,-0.204143,-2.314167,0.801532,-0.155248,0.520000,0.523706,-1.210116,-0.156799,-0.477957,1.038765,0.119195,0.488996,1.960678,1.142671,-0.747716,-2.024254
4,0.673338,-0.345606,0.246741,0.238186,0.948366,-0.041175,0.279368,0.034308,1.173739,2.115742,-2.095645,-1.049091,-0.909282,0.845759,0.035772,0.965730,2.139751,1.504896,-0.972603,-2.517911


In [18]:
not_saved_yet = False

if not_saved_yet:
    # ADD FULL PATHS TO THESE SO THEY DONT JUST SAVE IN THE REPO!
    ppd_emg_df.to_pickle('$BStand_EMG_df.pkl')
    X_ppd.to_pickle('$BStand_IMUEMG_df.pkl')
    df_t.to_pickle('$BStand_PCA40_IMUEMG_df.pkl')
    full_PCA20.to_pickle('$BStand_PCA20_IMUEMG_df.pkl')
    metadata_cols_df.to_pickle('metadata_cols_df.pkl')

## EMG Only

In [ ]:
#emg_file_path_kai = 'C:\\Users\\kdmen\\Desktop\\Research\\Data\\$M\\filtered_datasets\\metadata_EMG_allgestures_allusers.pkl'
#emg_file_path_brc = 'D:\\Kai_MetaGestureClustering_24\\saved_datasets\\filtered_datasets\\metadata_EMG_allgestures_allusers.pkl'
#emg_df = pd.read_pickle(emg_file_path_brc)
#ms_emg_df = meansubtract_df_by_gesture(emg_df)

# ALREADY HAVE ppd_emg_df!

In [19]:
emg_PCA3, dim_reduc_model3 = apply_dim_reduc(ppd_emg_df, model_str='PCA', num_dims=3, use_full_dataset=True)

Start
Success


In [20]:
emg_PCA8, dim_reduc_model8 = apply_dim_reduc(ppd_emg_df, model_str='PCA', num_dims=8, use_full_dataset=True)

Start
Success


In [24]:
emg_PCA8.head()

,0,1,2,3,4,5,6,7
0,-1.341323,-0.812209,0.089633,-0.231389,-1.102774,-0.123494,-0.452216,-0.563469
1,-0.644262,-0.860011,0.240803,-0.036121,-1.728382,-0.271787,-1.100263,0.143207
2,-1.251829,-0.823973,-0.213519,0.139521,-2.368628,-0.276726,-2.114243,0.468320
3,-1.494897,-0.633561,-0.233368,0.250943,-2.434067,-0.437012,-2.570415,0.370891
4,1.615750,-1.806795,-1.070433,-0.332640,-2.581807,-0.507581,-3.102347,0.600151


Save the above datasets

In [21]:
not_saved_yet = False

if not_saved_yet:
    emg_PCA3.to_pickle('$BStand_EMG_PCA3_df.pkl')
    emg_PCA8.to_pickle('$BStand_EMG_PCA8_df.pkl')

Now do train test split

In [29]:
def manual_train_test_split(df_t, metadata_cols_df, save_file_name, save_bool=True, save_path='D:\\Kai_MetaGestureClustering_24\\saved_datasets', user_holdout=True, gesture_holdout=False, held_out_user_pids=['P103','P109','P114','P124','P128','P005','P010'], held_out_test_gestures=['move', 'zoom-out', 'duplicate']):

    save_lst = []
    save_name_lst = []

    # Combine the data and metadata dfs again:
    # Ensure both DataFrames have the same index
    df_t.reset_index(drop=True, inplace=True)
    metadata_cols_df.reset_index(drop=True, inplace=True)
    # Concatenate the DataFrames
    metadata_PCA_df = pd.concat([metadata_cols_df, df_t], axis=1)
    save_lst.append(metadata_PCA_df)
    save_name_lst.append("full_dimreduc_df.pkl")

    if user_holdout:
        test_users_df = metadata_PCA_df[metadata_PCA_df['Participant'].isin(held_out_user_pids)]
        # Merge the DataFrames with an indicator
        merged_df = metadata_PCA_df.merge(test_users_df, how='left', indicator=True)
        # Filter out the rows that are in both DataFrames
        training_users_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns='_merge')
        save_lst.extend([test_users_df, training_users_df])
        #save_name_lst.extend([save_file_name+'_test_users_df.pkl', save_file_name+'_training_users_df.pkl'])
        save_name_lst.extend(['test_users_df.pkl', 'training_users_df.pkl'])
    if gesture_holdout:
        test_gestures_df = metadata_PCA_df[(metadata_PCA_df['Participant'].isin(held_out_user_pids)) & (data_df['Gesture_ID'].isin(held_out_gestures))]
        # Merge the DataFrames with an indicator
        merged_df = metadata_PCA_df.merge(test_gestures_df, how='left', indicator=True)
        # Filter out the rows that are in both DataFrames
        training_gestures_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns='_merge')
        save_lst.extend([test_gestures_df, training_gestures_df])
        #save_name_lst.extend([save_file_name+'_test_gestures_df.pkl', save_file_name+'_training_gestures_df.pkl'])
        save_name_lst.extend(['test_gestures_df.pkl', 'training_gestures_df.pkl'])
        
    if save_bool:
        for idx, ele in enumerate(save_lst):
            ele.to_pickle(save_path + '\\' + save_file_name + '\\' + save_name_lst[idx])

    return save_lst, save_name_lst

In [30]:
full_PCA40_return_lst, save_name_lst = manual_train_test_split(full_PCA40, metadata_cols_df, 'Both_PCA40', save_bool=True, save_path='D:\\Kai_MetaGestureClustering_24\\saved_datasets', user_holdout=True, gesture_holdout=False)


In [31]:
full_PCA20_return_lst, save_name_lst = manual_train_test_split(full_PCA20, metadata_cols_df, 'Both_PCA20', save_bool=True, save_path='D:\\Kai_MetaGestureClustering_24\\saved_datasets', user_holdout=True, gesture_holdout=False)


In [32]:
emg_PCA3_return_lst, save_name_lst = manual_train_test_split(emg_PCA3, metadata_cols_df, 'EMG_PCA3', save_bool=True, save_path='D:\\Kai_MetaGestureClustering_24\\saved_datasets', user_holdout=True, gesture_holdout=False)


In [33]:
emg_PCA8_return_lst = manual_train_test_split(emg_PCA8, metadata_cols_df, 'EMG_PCA8', save_bool=True, save_path='D:\\Kai_MetaGestureClustering_24\\saved_datasets', user_holdout=True, gesture_holdout=False)
